In [72]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langgraph langgraph-prebuilt langgraph_sdk langgraph-checkpoint-sqlite langsmith langchain-community tavily-python wikipedia

In [73]:
from dotenv import load_dotenv
load_dotenv()

True

In [74]:
import operator
from typing import List, Annotated, TypedDict
from langchain_core.messages import AnyMessage, HumanMessage
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages

# 1. Defining The State

class TravelAgentState(TypedDict):
    
    
    messages: Annotated[List[AnyMessage], add_messages] 
    
    
    destination: str
    budget: str
    travel_dates: str
    
    flight_options: Annotated[List[dict], operator.add]
    train_options: Annotated[List[dict], operator.add]
    bus_options: Annotated[List[dict], operator.add]
    accommodation_options: Annotated[List[dict], operator.add]


In [75]:
# 2. Setting Up Memory
import sqlite3
conn = sqlite3.connect(":memory:", check_same_thread = False)
from langgraph.checkpoint.sqlite import SqliteSaver
memory = SqliteSaver(conn)

In [76]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
import os

# 3. Defining The Tools
@tool
def search_flights(destination: str, dates: str, budget: str) -> List[dict]:
    """Searches for flights based on destination, dates, and budget."""
    print(f"--- TOOL: Searching flights to {destination} ---")
    return [{"type": "flight", "id": "fl_001", "name": "LangAir", "price": 450, "link": "https://example.com/flights/fl_001"}]
@tool
def search_trains(destination: str, dates: str) -> List[dict]:
    """Searches for trains based on destination and dates."""
    print(f"--- TOOL: Searching trains to {destination} ---")
    return [{"type": "train", "id": "trn_001", "name": "GraphRail", "price": 90, "link": "https://example.com/trains/trn_001"}]
@tool
def search_buses(destination: str, dates: str) -> List[dict]:
    """Searches for buses based on destination and dates."""
    print(f"--- TOOL: Searching buses to {destination} ---")
    return [{"type": "bus", "id": "bus_001", "name": "NodeExpress", "price": 50, "link": "https://example.com/buses/bus_001"}]
@tool
def search_hotels(destination: str, dates: str, budget: str) -> List[dict]:
    """Searches for hotels in the given destination for the given dates."""
    print(f"--- TOOL: Searching hotels in {destination} ---")
    return [{"type": "hotel", "id": "htl_001", "name": "The Checkpointer Inn", "price": 120, "link": "https://example.com/hotels/htl_001"}]
@tool
def search_airbnbs(destination: str, dates: str, budget: str) -> List[dict]:
    """Searches for Airbnbs in the given destination for the given dates."""
    print(f"--- TOOL: Searching Airbnbs in {destination} ---")
    return [{"type": "airbnb", "id": "ab_001", "name": "Cozy Loft by the Nodes", "price": 90, "link": "https://example.com/airbnb/ab_001"}]
tools = [search_flights, search_trains, search_buses, search_hotels, search_airbnbs]


# 4. Defining The Agent with System Message

llm = ChatOpenAI(model="gpt-4o-mini") 
llm_with_tools = llm.bind_tools(tools)

SYSTEM_MESSAGE = (
    "You are a helpful travel planning assistant. "
    "When a user provides their destination, dates, and budget, your goal is to find all available travel and accommodation options. "
    "When you have the results from your tools, present them clearly to the user in a list, including the type, name, price, and booking link. "
    "Also give them the total cost of their selected options. "
    "Do not make up any information that is not from your tools."
)



def assistant_node(state: TravelAgentState):
    """
    This is the main agent node. It calls the LLM, which can either
    respond directly to the user or decide to call one or more tools.
    """

    messages_with_system_prompt = [SystemMessage(content=SYSTEM_MESSAGE)] + state['messages']
    
    response = llm_with_tools.invoke(messages_with_system_prompt)
    
    return {"messages": [response]}

tool_node = ToolNode(tools)

In [77]:
# 5. Building the graph

builder = StateGraph(TravelAgentState)

builder.add_node("assistant", assistant_node)
builder.add_node("tools", tool_node)

builder.add_edge(START, "assistant")


builder.add_conditional_edges(
    "assistant",
    tools_condition,
    {
        "tools": "tools",
        END: END
    }
)

builder.add_edge("tools", "assistant")

app = builder.compile(checkpointer=memory)

In [78]:
# 6. Testing The Agent

config = {"configurable": {"thread_id": "user-conversation-2"}}

first_input = {"messages": [HumanMessage(content="Hi! I want to plan a trip to Tokyo.")]}
response = app.invoke(first_input, config)

response['messages'][-1].pretty_print()

second_input = {"messages": [HumanMessage(content="Can you find flights and a hotel? My budget is $2000 and dates are 2026-01-01 to 2026-01-08.")]}

for chunk in app.stream(second_input, config, stream_mode="values"):
    if "messages" in chunk:
        chunk['messages'][-1].pretty_print()

================================== Ai Message ==================================

Great choice! To help you plan your trip to Tokyo, could you please provide the following details?

1. Your travel dates (check-in and check-out).
2. Your budget for flights and accommodation.
================================ Human Message =================================

Can you find flights and a hotel? My budget is $2000 and dates are 2026-01-01 to 2026-01-08.
================================== Ai Message ==================================
Tool Calls:
  search_flights (call_YKsCPGQR1R8K5ljmccGeg4QE)
 Call ID: call_YKsCPGQR1R8K5ljmccGeg4QE
  Args:
    destination: Tokyo
    dates: 2026-01-01 to 2026-01-08
    budget: 2000
  search_hotels (call_UfpW4tUBV0p2vn4xdr69C2Ws)
 Call ID: call_UfpW4tUBV0p2vn4xdr69C2Ws
  Args:
    destination: Tokyo
    dates: 2026-01-01 to 2026-01-08
    budget: 2000
--- TOOL: Searching flights to Tokyo ---
--- TOOL: Searching hotels in Tokyo ---
===============================